<a href="https://imla.hs-offenburg.de/"> 
![hs logo](../images/hs-logo.png)  
![IMLA Banner](../images/imla-banner.png)
</a>

Welcome to the Summer School of the Hochschule Offenburg! Before we start, here are a few **notes** about Jupyter notebooks.

1. This Jupyter notebook is rendered in your browser, but the content is streamed by an interactive iPython kernel.

2. The notebook consists of cells; cells can contain code that you can run, or they can contain text and/or images that you can read.

3. You can execute code cells by clicking on the ```Run``` icon in the menu or by using the following key combinations ```Shift-Enter``` (execute and forward) or ```Ctrl-Enter``` (execute and stay in the current cell).

4. To interrupt cell execution, click the ```Stop``` button on the toolbar or navigate to the ```Kernel``` menu and select ```Interrupt Kernel```.

5. The button ```H``` gives you an overview of the configured keyboard shortcuts and information about the modes (Command Mode and Edit Mode) of Jupyter Notebook.

I am a text cell

In [ ]:
# I'm a code cell.
1 + 1 

In [ ]:
# with a prefixed '!' it is possible to execute Bash commands within code cells as well
!ls -la

# Deep Learning with Keras

[Keras](https://keras.io/) is a Python library that allows you to programmatically create and train artificial neural networks. Keras is an abstraction layer to existing Deep Learning Frameworks (Tensorflow, CNTK and Theano). It therefore uses the mentioned frameworks as backend.  
  
In this notebook we want to deal with the classification of the Mnist dataset, the Hello World of Deep Learning, and thus slowly familiarize ourselves with the Keras Framework. As at the beginning of every Python script, the Python modules to be used must be made known to the environment.

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential,load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import RMSprop
from keras import metrics as k_metrics
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.preprocessing import image as keras_image

from sklearn.metrics import roc_curve, auc, confusion_matrix


import time 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import glob

In [ ]:
from keras import backend as K
import tensorflow as tf
# For Jetson TX2!
# prevent Tensorflow to use all available memory 

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

## Constants

In the following, constants are defined, which determine the further execution. In addition, some hyperparameters are determined.

In [ ]:
DENSE_MODEL = 'MnistDenseClassifier.hdf5'
CNN_MODEL = 'MnistCnnClassifier.hdf5'
IMAGE_DIR = './data'

TARGET_SHAPE = (28, 28, 1)
NUM_CLASSES = 10

INVERSE_DATA = False # Don't use b/w inverse images
MODEL_TYPE = CNN_MODEL
BATCH_SIZE = 32
OPTIMIZER = RMSprop()
EPOCHS = 5

## Data Preparation
Now the images are loaded and normalized

In [ ]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if MODEL_TYPE is CNN_MODEL:
    x_train = x_train.reshape(60000, 28, 28, 1)
    x_test = x_test.reshape(10000, 28, 28, 1)
elif MODEL_TYPE is DENSE_MODEL:
    TARGET_SHAPE = (784,)
    x_train = x_train.reshape(60000, 784)
    x_test = x_test.reshape(10000, 784)

if INVERSE_DATA:
    # invert the data (black/white) and concatenate it to the existing date.
    x_train = np.concatenate((x_train, np.invert(x_train)))
    y_train = np.concatenate((y_train,y_train))
    x_test = np.concatenate((x_test, np.invert(x_test)))
    y_test = np.concatenate((y_test, y_test))

# normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

## Model Definition

Now the network is created depending on **MODEL_TYPE**. The _Sequential_ class shows that in both cases it is a simple _Feedforward Network_. After the model definition, the _compile_ method adds functions for _loss_, _optimizer_ and _metrics_ to the model for the training process.

In [ ]:
if MODEL_TYPE is DENSE_MODEL:
    model = Sequential([
        Dense(units=512, activation='relu', input_shape=TARGET_SHAPE),
        Dense(NUM_CLASSES, activation='softmax')
    ])

if MODEL_TYPE is CNN_MODEL:
    model = Sequential([
            Conv2D(filters=8, kernel_size=(3, 3), activation='relu', input_shape=TARGET_SHAPE),
            MaxPooling2D(pool_size=(3, 3)),
            Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dense(units=32, activation='relu'),
            Dense(units=NUM_CLASSES, activation='softmax')
    ])

# compile the model. 
model.compile(loss='categorical_crossentropy',
              optimizer=OPTIMIZER,
              metrics=[k_metrics.categorical_accuracy])

# print the models layout
model.summary()

## Model Training

Now we have the training data and a model ready. Before we do the training (_fit(..)_), we create a tensorboard callback, which logs the training process. We can view the progress by starting Tensorboard with `tensorboard --logdir=/path/to/logdir/`. After the successful start of Tensorboard it is accessible via port 6006 via the browser `localhost:6006`.  

In [ ]:
sub_dir = time.strftime("%Y_%m_%d-%H_%M_%S", time.localtime())
tb_cb = TensorBoard(log_dir='./logs/'+sub_dir,
                    #histogram_freq=EPOCHS,
                    batch_size=BATCH_SIZE, 
                    write_grads=True,
                    write_images=True,
                    #update_freq='batch',
                   )
callbacks = [tb_cb]

history = model.fit(x_train, y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_split=0.16666,             
                    callbacks=callbacks)

## Model Evaluation  

Now that we have a trained model, we want to determine its quality. The test data set is available in the variables x_train (images) and y_train (classes). With the method *evaluate(..)* we can finally determine the accuracy of the model.


### Accuracy

In [ ]:
(test_loss,test_acc) = model.evaluate(x_test,y_test, verbose=0)
print('TEST_LOSS: {:.3f}\nTEST_ACCURACY: {:.3f}'.format(test_loss,test_acc))

### Confusion Matrix & Classification Report
The Accuracy and the F1_Score were calculated again using the Machine Learning library [scikit-learn](https://scikit-learn.org/stable/index.html) as a proof for the model evaluation. In addition, the library offers methods to create a [confusion matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)  and [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html).

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score, f1_score

# make a prediction for the complete test set
prediction = model.predict(x_test, verbose=1)

y_true = np.argmax(np.squeeze(np.array(y_test)),axis=1)
# transform prediction to class label (for each image the class)
y_pred = np.argmax(np.squeeze(np.array(prediction)),axis=1)

# same score as with model.evaluate(..)?
print('Accuracy: {:.3f}'.format(accuracy_score(y_true,y_pred)))
print('F1_Score: {:.3f}'.format(f1_score(y_true,y_pred,average='weighted')))
display(confusion_matrix(y_true,y_pred))
print(classification_report(y_true,y_pred, digits=3))

### Plot of False Negative and False Positive

Below is a representation of the *False Negative* and *False Positive* in reference to the class `LABEL`.

In [ ]:
LABEL = 0

indices = np.arange(len(y_true))
indices = indices[y_true==LABEL]
true_positive_idx = indices[y_pred[indices]==LABEL]
false_negative_idx = indices[y_pred[indices]!=LABEL] 

indices = np.arange(len(y_true))
indices = indices[y_pred==LABEL]
false_positive_idx = indices[y_true[indices]!=LABEL]

def print_images(indices,suptitel):
    fig = plt.figure(figsize=(15,5))
    fig.suptitle(suptitel, fontsize=16)
    for i,idx in enumerate(indices[:20]):
        img = x_test[idx]
        img = np.reshape(img,(28,28))
        ax = plt.subplot(2,10,i+1)
        ax.imshow(img,cmap='gray')
        ax.set_title('True Class: {}\nPrediction: {}\nProp:{:.3f}'.format(y_true[idx],
                                                                          y_pred[idx],
                                                                          prediction[idx][y_pred[idx]] ))
        ax.tick_params(bottom=False, left=False,  labelleft=False, labelbottom=False)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)

print_images(false_negative_idx, 'False Negative')
print_images(false_positive_idx, 'False Positive')

## Prediction of own Numbers

Now that we have adapted the model to the training data, we want to test some of our own pictures. For this purpose there are predefined images under the directory `./data/`, which are loaded one after the other and submitted to the network for classification. The result image presents the individual results of each image. For each image the result shows the maximum value of the Softmax function and the predicted class (number).

In [ ]:
images = glob.glob(IMAGE_DIR + '/*.jpg')
images.sort()

COLS = 8
rows = len(images)//COLS+1
plt.figure(figsize=(15,4))

for idx, image in enumerate(images):
    
    ax = plt.subplot(rows,COLS,idx+1)
    
    img = keras_image.load_img(path=image, color_mode='grayscale', target_size=(28, 28, 1))
    img = keras_image.img_to_array(img)
    img2predict = img.copy()
    
    if MODEL_TYPE is DENSE_MODEL:
        img2predict = np.reshape(img,TARGET_SHAPE)
    img2predict = np.expand_dims(img2predict,0)
    img2predict /= 255
    
    pred = model.predict(img2predict)
    pred = np.ravel(pred)
    pred = np.round(pred, 3)
    
    class_predict = np.argmax(pred)
    # show the output of the model
    #print(image, pred)
    img = np.squeeze(img)
    ax.imshow(img,cmap='gray')
    ax.set_title('{}\n Prob: {:0.3f}\n Prediction: {}'.format(image,pred[class_predict],class_predict ))
    ax.tick_params(bottom=False, left=False,  labelleft=False, labelbottom=False)
    
plt.tight_layout()
plt.show()